# S-DES Algorithm
## - 32567 Harshal Tak
<h2 style="color:blue">- (original work) </h2>

## Required Permutations and Initial 10-bit Key

In [ ]:
# 10-bit key
key = [1, 0, 1, 0, 0, 0, 0, 0, 1, 0]
P_10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6 ] # P10 Table
P_8 = [6, 3, 7, 4, 8, 5, 10, 9] # P8 Table
P_4 = [2,4,3,1] # P4 Table
Perm = [2,6,3,1,4,8,5,7] # Initial Permutation
IP_1 = [4,1,3,5,7,2,8,6] # Inverse Permutation

In [ ]:
# Substitution tables
S0 = [[1,0,3,2],
      [3,2,1,0],
      [0,2,1,3],
      [3,1,3,2]]
S1=  [[0,1,2,3],
      [2,0,1,3],
      [3,0,1,0],
      [2,1,0,3]]

## Functions required in code

In [ ]:
def permute(table,key,permute_table):
    for i in range(len(permute_table)):
        index = permute_table[i]
        table[i] = key[index-1]
    return table

def Left_shift(lst):
    new_lst = [0,0,0,0,0]
    for i in range(0,5):
        if(i == 0):
            new_lst[4] = lst[0]
        else:
            new_lst[i-1] = lst[i]
    return new_lst

def bitshifting(bitlist):
    out = 0
    for bit in bitlist:
        out = (out << 1) | bit
    return out

def substitution(row,col,S):
    val = 0
    for i in range(len(S)):
        for j in range(len(S)):
            if(i==row and j==col):
                val = S[i][j]
    val = bin(val)[2:]
    if val == '0':
        val+='0'
    if val == '1':
        val= '0'+val
    val = [int(s) for s in [*val]]
    return val


## Key Generation Function (Creates two 8-bit keys)

In [ ]:
def key_generation(P_8,P_10,key):
    key_P10 = [0,0,0,0,0,0,0,0,0,0]
    print("P(10) Value: ",permute(key_P10,key,P_10))

    left = key_P10[0:5]
    right = key_P10[5:10]
    print("Left half: ",left)
    print("Right half: ",right)

    left1 = Left_shift(left)
    right1 = Left_shift(right)
    print("After 1-bit Left Shift: ")
    print("Left: ",left1)
    print("Right: ",right1)

    LS1 = left1+right1
    print("LS1: ",LS1)

    P8_key = LS1
    key_P8 = [0,0,0,0,0,0,0,0,0,0]
    key1 = permute(key_P8,P8_key,P_8)
    key1 = key1[0:8]

    left2 = Left_shift(left1)
    right2 = Left_shift(right1)
    left2 = Left_shift(left2)
    right2 = Left_shift(right2)
    print("2-bit Left Shift on 1-Left Shift one: ")
    print("Left: ",left2)
    print("Right: ",right2)

    LS2 = left2+right2
    print("LS2: ",LS2)

    P8_key2 = LS2
    key_P8 = [0,0,0,0,0,0,0,0,0,0]
    key2 = permute(key_P8,P8_key2,P_8)
    key2 = key2[0:8]

    return key1,key2

## Function which carries out expansion permutation, XOR operation and Substitution operations

In [ ]:
def complexfunc(key,left,right):
    # Expanded Permutation on right half
    indexr = [4,1,2,3,2,3,4,1]
    EP1 = []
    for i in range(len(indexr)):
        index = indexr[i]
        val = right[index-1]
        EP1.append(val)
    print("Expanded Permutation: ",EP1)

    # XOR on K1 and EP1
    XOR = []
    for i in range(0,8):
        result = key[i]^EP1[i]
        XOR.append(result)
    print("XOR between key and expandend permutation 1: ",XOR)

    lx = XOR[0:4]
    rx = XOR[4:8]
    print("Left XOR: ",lx)
    print("Right XOR: ",rx)

    # Substitution
    # For left:
    lrow =[lx[0],lx[3]]
    lcol =[lx[1],lx[2]]
    row_l = bitshifting(lrow)
    col_l = bitshifting(lcol)
    print("Row: ",row_l," Column: ",col_l)
    Subs_left = substitution(row_l,col_l,S0)
    print("Left 2-bits: ",Subs_left)

    # For right:
    rrow =[rx[0],rx[3]]
    rcol =[rx[1],rx[2]]
    row_r = bitshifting(rrow)
    col_r = bitshifting(rcol)
    print("Row: ",row_r," Column: ",col_r)
    Subs_right = substitution(row_r,col_r,S1)
    print("Right 2-bits: ",Subs_right)

    Subs1 = Subs_left+Subs_right
    print("Combined parts: ",Subs1)

    # P4 on Subs1
    result = [0,0,0,0,0,0,0,0]
    P4 = permute(result,Subs1,P_4)
    P4 = P4[0:4]
    print("P4 result: ",P4)

    # XOR P4 and left IP
    XOR2 = []
    for i in range(0,4):
        val = left[i]^P4[i]
        XOR2.append(val)
    print(f"XOR between P4 {P4} and left Initial Permutation {left} : ",XOR2)

    # Combine right IP and XOR2
    comb = XOR2+right
    return comb

## Encryption Function

In [ ]:
def encryption(plain_text,Perm,IP_1,key1,key2):

    result = [0,0,0,0,0,0,0,0]
    IP = permute(result,plain_text,Perm)
    print("Initial Permutation: ",IP)

    l = IP[0:4]
    r = IP[4:8]
    print("Left Half: ",l)
    print("Right Half: ",r)

    comb1 = complexfunc(key1,l,r)
    print("Comb1: ",comb1)
    combl = comb1[0:4]
    combr = comb1[4:8]
    comb1 = combr+combl
    print("Join right half then left half: ",comb1)
    l1 = comb1[0:4]
    r1 = comb1[4:8]

    comb2 = complexfunc(key2,l1,r1)
    print("Comb2: ",comb2)

    result = [0,0,0,0,0,0,0,0]
    cipher = permute(result,comb2,IP_1)

    return cipher

## Decryption Function

In [ ]:
def decryption(cipher,Perm,IP_1,key1,key2):
    result = [0,0,0,0,0,0,0,0]
    IP = permute(result,cipher,Perm)
    print("Initial Permutation: ",IP)

    l = IP[0:4]
    r = IP[4:8]
    print("Left Half: ",l)
    print("Right Half: ",r)

    comb1 = complexfunc(key2,l,r)
    print("Comb1: ",comb1)
    combl = comb1[0:4]
    combr = comb1[4:8]
    comb1 = combr+combl
    print("Join right half then left half: ",comb1)
    l1 = comb1[0:4]
    r1 = comb1[4:8]

    comb2 = complexfunc(key1,l1,r1)
    print("Comb2: ",comb2)

    result = [0,0,0,0,0,0,0,0]
    decrypted = permute(result,comb2,IP_1)

    return decrypted

## Input plain text

In [ ]:
plain_text = [1,0,1,1,0,1,0,0]
key1,key2 = key_generation(P_8,P_10,key)
cipher = encryption(plain_text,Perm,IP_1,key1,key2)

P(10) Value:  [1, 0, 0, 0, 0, 0, 1, 1, 0, 0]
Left half:  [1, 0, 0, 0, 0]
Right half:  [0, 1, 1, 0, 0]
After 1-bit Left Shift: 
Left:  [0, 0, 0, 0, 1]
Right:  [1, 1, 0, 0, 0]
LS1:  [0, 0, 0, 0, 1, 1, 1, 0, 0, 0]
2-bit Left Shift on 1-Left Shift one: 
Left:  [0, 0, 1, 0, 0]
Right:  [0, 0, 0, 1, 1]
LS2:  [0, 0, 1, 0, 0, 0, 0, 0, 1, 1]
Initial Permutation:  [0, 1, 1, 1, 1, 0, 0, 1]
Left Half:  [0, 1, 1, 1]
Right Half:  [1, 0, 0, 1]
Expanded Permutation:  [1, 1, 0, 0, 0, 0, 1, 1]
XOR between key and expandend permutation 1:  [0, 1, 1, 0, 0, 1, 1, 1]
Left XOR:  [0, 1, 1, 0]
Right XOR:  [0, 1, 1, 1]
Row:  0  Column:  3
Left 2-bits:  [1, 0]
Row:  1  Column:  3
Right 2-bits:  [1, 1]
Combined parts:  [1, 0, 1, 1]
P4 result:  [0, 1, 1, 1]
XOR between P4 [0, 1, 1, 1] and left Initial Permutation [0, 1, 1, 1] :  [0, 0, 0, 0]
Comb1:  [0, 0, 0, 0, 1, 0, 0, 1]
Join right half then left half:  [1, 0, 0, 1, 0, 0, 0, 0]
Expanded Permutation:  [0, 0, 0, 0, 0, 0, 0, 0]
XOR between key and expandend permuta

In [ ]:
decrypted = decryption(cipher,Perm,IP_1,key1,key2)

Initial Permutation:  [0, 0, 0, 0, 0, 0, 0, 0]
Left Half:  [0, 0, 0, 0]
Right Half:  [0, 0, 0, 0]
Expanded Permutation:  [0, 0, 0, 0, 0, 0, 0, 0]
XOR between key and expandend permutation 1:  [0, 1, 0, 0, 0, 0, 1, 1]
Left XOR:  [0, 1, 0, 0]
Right XOR:  [0, 0, 1, 1]
Row:  0  Column:  2
Left 2-bits:  [1, 1]
Row:  1  Column:  1
Right 2-bits:  [0, 0]
Combined parts:  [1, 1, 0, 0]
P4 result:  [1, 0, 0, 1]
XOR between P4 [1, 0, 0, 1] and left Initial Permutation [0, 0, 0, 0] :  [1, 0, 0, 1]
Comb1:  [1, 0, 0, 1, 0, 0, 0, 0]
Join right half then left half:  [0, 0, 0, 0, 1, 0, 0, 1]
Expanded Permutation:  [1, 1, 0, 0, 0, 0, 1, 1]
XOR between key and expandend permutation 1:  [0, 1, 1, 0, 0, 1, 1, 1]
Left XOR:  [0, 1, 1, 0]
Right XOR:  [0, 1, 1, 1]
Row:  0  Column:  3
Left 2-bits:  [1, 0]
Row:  1  Column:  3
Right 2-bits:  [1, 1]
Combined parts:  [1, 0, 1, 1]
P4 result:  [0, 1, 1, 1]
XOR between P4 [0, 1, 1, 1] and left Initial Permutation [0, 0, 0, 0] :  [0, 1, 1, 1]
Comb2:  [0, 1, 1, 1, 1, 0, 

## Output

In [ ]:
print("Key1: ",key1)
print("Key2: ",key2)
print("Plain text: ",plain_text)
print("Cipher Text: ",cipher)
print("Decrypted text: ",decrypted)

Key1:  [1, 0, 1, 0, 0, 1, 0, 0]
Key2:  [0, 1, 0, 0, 0, 0, 1, 1]
Plain text:  [1, 0, 1, 1, 0, 1, 1, 0]
Cipher Text:  [0, 0, 0, 0, 0, 0, 0, 0]
Decrypted text:  [1, 0, 1, 1, 0, 1, 1, 0]
